# 🎓 Phiversity - Deploy in Google Colab
Deploy the Phiversity physics animation project directly in Google Colab with AI-powered video generation!

## 1️⃣ Mount Google Drive
Mount your Google Drive to save and access files persistently across sessions.

In [ ]:
from google.colab import drive
import os

# Mount Google Drive
drive.mount('/content/drive')
print("✅ Google Drive mounted at /content/drive")

## 2️⃣ Clone Repository from GitHub
Clone the Phiversity repository and navigate to the project directory.

In [ ]:
import subprocess
import os

# Clone the repository
repo_url = "https://github.com/sudish80/Phiversity.git"
project_dir = "/content/Phiversity"

if not os.path.exists(project_dir):
    print(f"Cloning repository from {repo_url}...")
    subprocess.run(["git", "clone", repo_url, project_dir], check=True)
    print("✅ Repository cloned successfully")
else:
    print(f"✅ Repository already exists at {project_dir}")

# Change to project directory
os.chdir(project_dir)
print(f"📂 Working directory: {os.getcwd()}")

## 3️⃣ Install Dependencies
Install all required packages for Manim, voice synthesis, LLM integration, and FastAPI.

In [ ]:
import subprocess
import sys

print("📦 Installing system dependencies...")
# Install system packages needed for Manim, ffmpeg, LaTeX, etc.
subprocess.run(["apt-get", "update"], check=True, capture_output=True)
subprocess.run([
    "apt-get", "install", "-y",
    "ffmpeg",
    "texlive-latex-base",
    "texlive-fonts-recommended",
    "texlive-latex-extra",
    "libcairo2-dev",
    "libpango1.0-dev",
    "espeak-ng"
], check=True, capture_output=True)
print("✅ System dependencies installed")

print("\n📦 Installing Python packages...")
# Install Python requirements
with open("requirements.txt", "r") as f:
    requirements = [line.strip() for line in f if line.strip() and not line.startswith("#")]

# Install in batches to avoid timeouts
subprocess.run([sys.executable, "-m", "pip", "install", "--upgrade", "pip"], check=True, capture_output=True)
subprocess.run([sys.executable, "-m", "pip", "install", "--no-cache-dir"] + requirements, check=True)
print("✅ Python dependencies installed")

## 4️⃣ Configure Environment Variables
Set up API keys and configuration for LLM providers and other services.

⚠️ **Important**: Replace the API keys below with your own from:
- **Groq**: https://console.groq.com
- **OpenRouter**: https://openrouter.ai
- **Gemini**: https://makersuite.google.com/app/apikey
- **Elevenlabs**: https://elevenlabs.io

In [ ]:
import os
from getpass import getpass

# LLM Configuration
print("🔧 Setting up environment variables...\n")

# Primary LLM Model (options: groq, openai, deepseek, gemini, ollama, openrouter)
os.environ["LLM_MODEL"] = "groq"
os.environ["USE_FINETUNED_MODEL"] = "true"

# Groq API Key
groq_key = getpass("Enter your GROQ_API_KEY (leave empty to use existing): ")
if groq_key:
    os.environ["GROQ_API_KEY"] = groq_key

# Gemini API Key (optional)
gemini_key = getpass("Enter your GEMINI_API_KEY (leave empty to skip): ")
if gemini_key:
    os.environ["GEMINI_API_KEY"] = gemini_key
    os.environ["GOOGLE_API_KEY"] = gemini_key

# OpenRouter API Key (optional)
openrouter_key = getpass("Enter your OPENROUTER_API_KEY (leave empty to skip): ")
if openrouter_key:
    os.environ["OPENROUTER_API_KEY"] = openrouter_key
    os.environ["OPENROUTER_MODEL"] = "anthropic/claude-3-sonnet"

# Voice Configuration
os.environ["VOICE_ENGINE"] = "pyttsx3"  # Options: pyttsx3, gtts, elevenlabs
os.environ["ELEVENLABS_API_KEY"] = ""  # Add if using ElevenLabs

# Manim Configuration
os.environ["VIDEO_QUALITY"] = "low_quality"  # low_quality, medium_quality, high_quality
os.environ["MANIM_QUALITY"] = "low_quality"

# Application Configuration
os.environ["OUTPUT_DIR"] = "media"
os.environ["GENERATE_AUDIO"] = "true"
os.environ["GENERATE_VIDEO"] = "true"
os.environ["JOB_TIMEOUT"] = "600"

print("✅ Environment variables configured")

## 5️⃣ Test LLM Connection
Verify that the LLM API connection is working correctly.

In [ ]:
import os
from pathlib import Path

# Test Groq API
print("🧪 Testing LLM connection with Groq...\n")

try:
    import groq
    
    groq_api_key = os.getenv("GROQ_API_KEY")
    if not groq_api_key:
        print("⚠️  GROQ_API_KEY not set. Please set it in the environment variables cell.")
    else:
        client = groq.Groq(api_key=groq_api_key)
        
        # Test simple message
        message = client.chat.completions.create(
            messages=[{"role": "user", "content": "Explain conservation of energy in one sentence."}],
            model="mixtral-8x7b-32768"
        )
        
        print("✅ Groq API Connection Successful!")
        print(f"\nSample response:\n{message.choices[0].message.content}\n")
        
except Exception as e:
    print(f"❌ Error testing Groq API: {e}")
    print("Please check your API key and try again.")

## 6️⃣ Run the Application
Execute the Phiversity application with the FastAPI server and generate physics animations.

The server will run on `http://localhost:8000`

In [ ]:
import subprocess
import time
import os
from pathlib import Path

# Change to project directory
os.chdir("/content/Phiversity")

# Install ngrok for tunneling (make the server accessible from outside)
print("📦 Installing ngrok for public URL access...")
subprocess.run(["pip", "install", "-q", "pyngrok"], check=True)

# Generate sample problem to test
sample_problem = "Explain how the conservation of momentum applies to a collision between two billiard balls. Show the before and after momentum vectors."

print("\n🚀 Starting Phiversity FastAPI Server...")
print(f"📝 Sample problem: {sample_problem}\n")

# Create a test script
test_script = '''
import os
import sys
import asyncio
from pathlib import Path

# Set environment
os.environ["LLM_MODEL"] = os.getenv("LLM_MODEL", "groq")
os.environ["MANIM_QUALITY"] = "low_quality"

# Add project to path
sys.path.insert(0, "/content/Phiversity")

# Import the orchestrator
from scripts.orchestrator.prompt_orchestrator import call_groq_solver

print("🧠 Processing physics problem with LLM...")
print(f"Problem: {problem}")

try:
    result = call_groq_solver(problem)
    print("\\n✅ LLM Processing Complete!")
    print(f"\\nGenerated Solution:\\n{result}")
    
    # Save result
    output_dir = Path("media/texts")
    output_dir.mkdir(parents=True, exist_ok=True)
    output_file = output_dir / "solution.txt"
    with open(output_file, "w") as f:
        f.write(result)
    print(f"\\n💾 Solution saved to {output_file}")
    
except Exception as e:
    print(f"\\n❌ Error: {e}")
    import traceback
    traceback.print_exc()
'''

# Write and run test script
with open("/tmp/test_phiversity.py", "w") as f:
    f.write(test_script.replace("{problem}", f'"{sample_problem}"'))

print("Running test...")
result = subprocess.run(
    ["python", "/tmp/test_phiversity.py"],
    cwd="/content/Phiversity",
    capture_output=True,
    text=True,
    timeout=300
)

print(result.stdout)
if result.stderr:
    print("Errors/Warnings:")
    print(result.stderr)

## 7️⃣ Download Output Files
Download generated videos, audio files, and text solutions to your local machine or Google Drive.

In [ ]:
import os
import shutil
from pathlib import Path
from google.colab import files

print("📁 Checking generated output files...\n")

media_dir = Path("/content/Phiversity/media")
output_files = {
    "Videos": list(media_dir.glob("videos/**/*.mp4")),
    "Audio": list(media_dir.glob("videos/**/*.wav")) + list(media_dir.glob("videos/**/*.mp3")),
    "Text Solutions": list(media_dir.glob("texts/**/*.json")) + list(media_dir.glob("texts/**/*.txt"))
}

# Display available files
for category, files_list in output_files.items():
    if files_list:
        print(f"✅ {category}:")
        for f in files_list[:5]:  # Show first 5 files
            file_size = f.stat().st_size / (1024*1024)  # Convert to MB
            print(f"   - {f.name} ({file_size:.2f} MB)")
        if len(files_list) > 5:
            print(f"   ... and {len(files_list) - 5} more files")
    else:
        print(f"⚠️  No {category.lower()} generated yet")

# Option to download files
print("\n💾 Downloading files to local machine...")
output_zip = "/tmp/phiversity_outputs.zip"

# Create zip file with all outputs
if list(media_dir.glob("**/*")):
    print("Creating archive...")
    shutil.make_archive("/tmp/phiversity_outputs", "zip", media_dir)
    print(f"✅ Archive created: {output_zip}")
    
    # Download
    print("Downloading to your computer...")
    files.download(output_zip)
    print("✅ Download complete!")
else:
    print("⚠️  No output files to download yet. Run the application first.")

## 🚀 BONUS: Run FastAPI Server (For Continuous Use)
Start the FastAPI server with public URL access via ngrok (requires ngrok authtoken).

In [ ]:
import subprocess
import os
from getpass import getpass
from pyngrok import ngrok

# Optional: Get ngrok authtoken from https://dashboard.ngrok.com/auth
print("🔐 Setting up public URL access with ngrok...\n")
print("To enable public access:")
print("1. Visit https://dashboard.ngrok.com/auth")
print("2. Copy your authtoken")
print("3. Paste it below (leave empty to skip)\n")

ngrok_token = getpass("Enter your ngrok authtoken (or press Enter to skip): ")

if ngrok_token:
    ngrok.set_auth_token(ngrok_token)
    print("✅ ngrok configured")
else:
    print("⚠️  Skipping ngrok - server will only be accessible locally")

print("\n🚀 Starting FastAPI Server...\n")

# Start the server in background
server_process = subprocess.Popen(
    ["python", "-m", "uvicorn", "scripts.server.app:app", "--host", "0.0.0.0", "--port", "8000"],
    cwd="/content/Phiversity",
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE
)

import time
time.sleep(5)  # Give server time to start

if ngrok_token:
    try:
        # Create ngrok tunnel
        public_url = ngrok.connect(8000)
        print(f"✅ Server running at {public_url}")
        print(f"\n📡 Public API URL: {public_url}/docs")
        print("🔗 You can now make requests to the API from anywhere!")
    except Exception as e:
        print(f"⚠️  ngrok error: {e}")
        print("Server is still running locally at http://localhost:8000")
else:
    print("✅ Server running locally at http://localhost:8000/docs")

print("\n⏱️  Server will keep running. You can now make API requests!")
print("To stop the server, run: server_process.terminate()")

## 📚 Troubleshooting & Next Steps

### Common Issues

**❌ "GROQ_API_KEY not found"**
- Solution: Get your API key from https://console.groq.com
- Re-run the environment variables cell and paste your key

**❌ "ModuleNotFoundError"**
- Solution: Make sure all dependencies are installed (run the installation cell again)
- Restart the kernel: Kernel → Restart Runtime

**❌ "Error generating video"**
- Check that you have enough Colab resources (runtime may timeout on complex videos)
- Use `VIDEO_QUALITY=low_quality` for faster generation

**❌ "ngrok not working"**
- Get a free account at https://ngrok.com
- Copy your authtoken from https://dashboard.ngrok.com/auth

### Next Steps

1. ✅ All files are already committed to GitHub
2. 📝 Modify `os.environ["LLM_MODEL"]` to try different LLMs
3. 🎬 Experiment with different physics problems
4. 📊 Save outputs to Google Drive for persistent storage
5. 🌐 Share the ngrok URL with collaborators for live demos

### Useful API Endpoints

- `GET /docs` - Interactive API documentation (Swagger UI)
- `POST /run` - Generate animation for a physics problem
- `POST /test-llm` - Test LLM connection with a question

### Resources

- 📖 [Phiversity GitHub](https://github.com/sudish80/Phiversity)
- 🎓 [Manim Documentation](https://docs.manim.community/)
- 🤖 [Groq API Docs](https://console.groq.com/docs)
- 🌐 [FastAPI Docs](https://fastapi.tiangolo.com/)